# gensim主题模型

In [1]:
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
filelist=[]
for i in range(len(df2)):
    filelist.append(df2['fenci'][i])
#生成文档对应的字典和bow稀疏矩阵
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist]

from gensim.models.ldamodel import LdaModel

#----------------------------------test----------------------------
n_topics = range(5, 75, 5)
perplexityLst = [1.0]*len(n_topics)
#训练LDA并打印训练时间
lda_models = []
for idx, n_topic in enumerate(n_topics):
    #lda = LatentDirichletAllocation(n_topics=n_topic,max_iter=20,learning_method='batch',evaluate_every=200,verbose=0)
    lda = LdaModel(corpus, id2word = dictionary, num_topics = n_topic, passes = 10)
    lda[corpus]
    
    perplexityLst[idx] = lda.log_perplexity(corpus)
    
    lda_models.append(lda)
    print("# of Topic: %d, " % n_topics[idx])
    print("Perplexity Score %0.3f" % perplexityLst[idx])
#打印最佳模型
best_index = perplexityLst.index(min(perplexityLst))
best_n_topic = n_topics[best_index]
best_model = lda_models[best_index]
print("Best # of Topic: ", best_n_topic)



#-----------------------------------------------------------------------

"""
# 列出所消耗的时间备查
%time ldamodel = LdaModel(corpus, id2word = dictionary, num_topics = 30, passes = 10) #拟合LDA模型
print(ldamodel.print_topics(num_topics = 30,num_words = 10))

# 计算各语料的LDA模型值
corpus_lda = ldamodel[corpus] # 此处应当使用和模型训练时相同类型的矩阵
for doc in corpus_lda:
    print(doc)

from gensim import similarities
simmtx = similarities.MatrixSimilarity(corpus) # 使用的矩阵种类需要和拟合模型时相同
simmtx.index[:2]

# 使用gensim的LDA拟合结果进行演示
query_bow = dictionary.doc2bow(df2.fenci[40])
print(df2.filename[0])
lda_vec = ldamodel[query_bow] # 转换为lda模型下的向量
sims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

for i in range(len(sims)):
    print(sims[i])
    a,b = sims[i]
    if b != 0.0:
        print(df.filename[a])
        
"""

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.037 seconds.
Prefix dict has been built succesfully.
C:\Tools\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# of Topic: 5, 
Perplexity Score -8.174
# of Topic: 10, 
Perplexity Score -8.033
# of Topic: 15, 
Perplexity Score -8.041
# of Topic: 20, 
Perplexity Score -7.998
# of Topic: 25, 
Perplexity Score -7.889
# of Topic: 30, 
Perplexity Score -7.936
# of Topic: 35, 
Perplexity Score -7.991
# of Topic: 40, 
Perplexity Score -7.912
# of Topic: 45, 
Perplexity Score -7.907
# of Topic: 50, 
Perplexity Score -7.953
# of Topic: 55, 
Perplexity Score -7.961
# of Topic: 60, 
Perplexity Score -7.995
# of Topic: 65, 
Perplexity Score -7.964
# of Topic: 70, 
Perplexity Score -8.066
Best # of Topic:  5


'\n# 列出所消耗的时间备查\n%time ldamodel = LdaModel(corpus, id2word = dictionary, num_topics = 30, passes = 10) #拟合LDA模型\nprint(ldamodel.print_topics(num_topics = 30,num_words = 10))\n\n# 计算各语料的LDA模型值\ncorpus_lda = ldamodel[corpus] # 此处应当使用和模型训练时相同类型的矩阵\nfor doc in corpus_lda:\n    print(doc)\n\nfrom gensim import similarities\nsimmtx = similarities.MatrixSimilarity(corpus) # 使用的矩阵种类需要和拟合模型时相同\nsimmtx.index[:2]\n\n# 使用gensim的LDA拟合结果进行演示\nquery_bow = dictionary.doc2bow(df2.fenci[40])\nprint(df2.filename[0])\nlda_vec = ldamodel[query_bow] # 转换为lda模型下的向量\nsims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询\nsims = sorted(enumerate(sims), key=lambda item: -item[1])\nsims\n\nfor i in range(len(sims)):\n    print(sims[i])\n    a,b = sims[i]\n    if b != 0.0:\n        print(df.filename[a])\n        \n'

In [10]:
import sys
import math
#reload(sys)
#sys.setdefaultencoding('utf-8')
import os
from gensim.corpora import Dictionary
from gensim import corpora, models
from datetime import datetime
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s : ', level=logging.INFO)
 
def perplexity(ldamodel, testset, dictionary, size_dictionary, num_topics):
    """calculate the perplexity of a lda-model"""
    # dictionary : {7822:'deferment', 1841:'circuitry',19202:'fabianism'...]
    print ('the info of this ldamodel: \n')
    print ('num of testset: %s; size_dictionary: %s; num of topics: %s'%(len(testset), size_dictionary, num_topics))
    prep = 0.0
    prob_doc_sum = 0.0
    topic_word_list = [] # store the probablity of topic-word:[(u'business', 0.010020942661849608),(u'family', 0.0088027946271537413)...]
    for topic_id in range(num_topics):
        topic_word = ldamodel.show_topic(topic_id, size_dictionary)
        dic = {}
        for word, probability in topic_word:
            dic[word] = probability
        topic_word_list.append(dic)
    doc_topics_ist = [] #store the doc-topic tuples:[(0, 0.0006211180124223594),(1, 0.0006211180124223594),...]
    for doc in testset:
        doc_topics_ist.append(ldamodel.get_document_topics(doc, minimum_probability=0))
    testset_word_num = 0
    for i in range(len(testset)):
        prob_doc = 0.0 # the probablity of the doc
        doc = testset[i]
        doc_word_num = 0 # the num of words in the doc
        for word_id, num in doc:
            prob_word = 0.0 # the probablity of the word 
            doc_word_num += num
            word = dictionary[word_id]
            for topic_id in range(num_topics):
                # cal p(w) : p(w) = sumz(p(z)*p(w|z))
                prob_topic = doc_topics_ist[i][topic_id][1]
                prob_topic_word = topic_word_list[topic_id][word]
                prob_word += prob_topic*prob_topic_word
            prob_doc += math.log(prob_word) # p(d) = sum(log(p(w)))
        prob_doc_sum += prob_doc
        testset_word_num += doc_word_num
    prep = math.exp(-prob_doc_sum/testset_word_num) # perplexity = exp(-sum(p(d)/sum(Nd))
    print ("the perplexity of this ldamodel is : %s"%prep)
    return prep
 
if __name__ == '__main__':
    
    from MyDataFrame import MyDataFrame
    mdf = MyDataFrame()
    df = mdf.new_DataFrame()
    df2 = mdf.m_cut(df)
    filelist=[]
    for i in range(len(df2)):
        filelist.append(df2['fenci'][i])
#生成文档对应的字典和bow稀疏矩阵
    from gensim import corpora, models  

    dictionary = corpora.Dictionary(filelist)  
    corpus = [dictionary.doc2bow(text) for text in filelist]
    
    from sklearn.externals import joblib
    lda_multi = joblib.load('ldamodel_0506_10.model')
    
    
    num_topics = 10
    testset = []
    # sample 1/300
    for i in range(int(len(df)/50)):
        testset.append(corpus[i*50])
        
    prep = perplexity(lda_multi, testset, dictionary, len(dictionary.keys()), num_topics)


2019-05-06 16:54:23,084 : INFO : adding document #0 to Dictionary(0 unique tokens: []) : 
2019-05-06 16:54:23,185 : INFO : built Dictionary(15144 unique tokens: ['0.01', '0.05', '0002', '00100.0095', '00105.00']...) from 50 documents (total 73016 corpus positions) : 


the info of this ldamodel: 

num of testset: 1; size_dictionary: 15144; num of topics: 10
the perplexity of this ldamodel is : 39.007824767774764
